#### 1. 安装langchain

In [3]:
!pip install langchain chromadb pymupdf

#### 2. 下载文档

In [18]:
PDF_NAME = 'C:\\Users\\Administrator\\Desktop\\OpsAny.pdf'

#### 3. 加载文档

In [19]:
from langchain.document_loaders import PyMuPDFLoader
docs = PyMuPDFLoader(PDF_NAME).load()
print(docs)

[Document(page_content='OpsAny 社区版提供了部署脚本进行快速安装，如果严格根据官方文档提示操作，可以保证100%安\n装成功率。\n部署导航\n\uf0b7\n在线部署手册\n\uf0b7\n基础监控部署\n\uf0b7\n应用平台部署\n\uf0b7\n离线部署手册\n部署逻辑介绍\n建议工程师阅读部署的Shell 脚本可以掌握部署的细节，\nOpsAny 采用Docker 容器化部署，\n所以部署\n的本质包含以下几个部分：\n\uf0b7\n1.自动为每个平台使用uuid 命令生成随机的APPID，根据install.config 配置批量修改配置\n文件。\n\uf0b7\n2.执行脚本将SaaS 应用注册到PaaS 平台上。\n\uf0b7\n3.拉取镜像并启动容器，挂载配置目录，挂载日志目录，挂载uploads 目录等。\n\uf0b7\n4.执行数据库同步\n', metadata={'source': 'C:\\Users\\Administrator\\Desktop\\OpsAny.pdf', 'file_path': 'C:\\Users\\Administrator\\Desktop\\OpsAny.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': 'Terry', 'subject': '', 'keywords': '', 'creator': 'WPS 文字', 'producer': '', 'creationDate': "D:20240410154202+07'42'", 'modDate': "D:20240410154202+07'42'", 'trapped': ''})]


#### 4. 拆分文档并存储文本嵌入的向量数据

In [20]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [22]:
split_docs = text_splitter.split_documents(docs)

In [23]:
embeddings = OllamaEmbeddings(model='nomic-embed-text')

In [24]:
vectorstore = Chroma.from_documents(split_docs, embeddings, collection_name="serverless_guide")

#### 5. 基于ollama创建QA链

In [25]:
from langchain_community.llms import Ollama
from langchain.chains.question_answering import load_qa_chain
llm = Ollama(model="qwen:7b")
chain = load_qa_chain(llm, chain_type="stuff")

#### 6. 基于提问，进行相似性查询

In [32]:
query = "opsany如何用docker部署?"
similar_docs = vectorstore.similarity_search(query)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


#### 7. 基于相似性查询结果，进行回答

In [38]:
chain.run(input_documents=similar_docs, question=query)

'OpsAny使用Docker进行容器化部署的流程如下：\n\n1. **自动生成APPID**：通过shell脚本，自动为每个平台使用`uuid`命令生成随机的APPID。同时根据`install.config`配置批量修改配置文件。\n\n2. **注册到PaaS平台上**：执行脚本将SaaS应用注册到PaaS平台上，确保服务的接入和管理。\n\n3. **拉取镜像并启动容器**：从Docker仓库中拉取相应的镜像，并启动容器。挂载必要的配置目录（如app.conf），日志目录，以及用户上传文件的uploads目录等。\n\n4. **执行数据库同步**：在容器启动后，确保进行数据库的相关操作，如数据迁移、备份恢复等。\n\n总结，OpsAny使用Docker进行部署的关键在于自动化脚本的编写和执行，以及对容器环境挂载相关资源的支持。\n'